In [ ]:
pip install nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/AI_CHATBOT/chatbot_model.h5') #Copy the path of your model which is uploaded on your google drive
import json
import random
intents = json.loads(open('/content/drive/MyDrive/AI_CHATBOT/intents.json', encoding="utf8").read()) #Copy the path of your json file which is uploaded on your google drive
words = pickle.load(open('/content/drive/MyDrive/AI_CHATBOT/words.pkl','rb')) #Copy the path of your pkl file which is uploaded on your google drive
classes = pickle.load(open('/content/drive/MyDrive/AI_CHATBOT/classes.pkl','rb')) #Copy the path of your pkl file which is uploaded on your google drive

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence words ", sentence_words)
    return sentence_words

In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    try:
      res = getResponse(ints, intents)
    except:
      print("An exception occurred")
      res = "I can't answer this query. I am sorry but this is out of my limitation."

    return res

In [ ]:
pip install flask-ngrok

In [ ]:
pip install Flask-Cors

In [ ]:
import logging
from flask import Flask, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok

app = Flask(__name__)

CORS(app)
run_with_ngrok(app)

@app.route("/",methods=["GET"])
def hello():
  return jsonify({"health":"server is running successfully"})

def decrypt(msg):
  string = msg
  new_string = string.replace("+"," ")
  return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
  dec_msg = decrypt(sentence)
  response = chatbot_response(dec_msg)
  json_obj = jsonify({"top" : {"res" : response}})
  return json_obj

if __name__ == '__main__':
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d3d8-34-106-175-95.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 13:36:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 13:36:36] "GET / HTTP/1.1" 200 -


sentence words  ['hey']
1/1 [==============================] - 0s 118ms/step


...

INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 13:50:27] "GET /query/Snacks+for+weight+loss+ HTTP/1.1" 200 -


sentence words  ['healthy', 'lifestyle']
1/1 [==============================] - 0s 28ms/step


INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 13:50:42] "GET /query/Healthy+lifestyle HTTP/1.1" 200 -


sentence words  ['home', 'remedy', 'for', 'managing', 'headache']
1/1 [==============================] - 0s 21ms/step


INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 13:50:55] "GET /query/Home+remedies+for+managing+headaches HTTP/1.1" 200 -
